In [1]:
import numpy as np
import pandas as pd

from collections import Counter
from typing import (
    List,
    Dict,
    Optional,
)
import re

import matplotlib.pyplot as plt
%matplotlib inline


Constants

In [2]:
RANDOM_SEED = 5
DATA_PATH = "raw_data"
AK_FILE = "AnnaKarenina"
WNP_RU_FILE = "WarAndPeace"
WNP_EN_FILE = "WarAndPeaceEng"
TXT = ".txt"

np.random.seed(RANDOM_SEED)


Data loading

In [3]:
wnp_ru = None
with open(f"{DATA_PATH}/{WNP_RU_FILE}{TXT}", "r", encoding="utf-8") as f:
    wnp_ru = f.read()
assert wnp_ru is not None


wnp_eng = None
with open(f"{DATA_PATH}/{WNP_EN_FILE}{TXT}", "r", encoding="utf-8") as f:
    wnp_eng = f.read()
assert wnp_eng is not None


ak = None
with open(f"{DATA_PATH}/{AK_FILE}{TXT}", "r", encoding="utf-8") as f:
    ak = f.read()
assert ak is not None


# 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.


In [9]:
class TextPreproccesing:
    def __init__(self, text: str) -> None:
        try:
            self.regex = re.compile("[\W_\d]+|[^А-Яа-я]+")
            self.original_text = self.regex.sub(" ", text.lower()).strip()
        except BaseException as e:
            print(f"error with text processing")
            print(e)

    def count_letters_freq(self):
        self.counter: Optional[Counter] = Counter(self.original_text)
        self.letter_num = len(self.counter.keys()) # 33 буквы
        return self

    #def decode(self, )


In [11]:
%%time


wnp_ru_processed = TextPreproccesing(wnp_ru).count_letters_freq()
print(wnp_ru_processed.letter_num)
print(wnp_ru_processed.original_text[:100*4])
print(wnp_ru_processed.counter)


33
война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философии эта книга из разряда вечных потому что она обо всем о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мире первый том знакомит с высшим обществом россии  века показаны взаимоотношения между родителями и детьми в семье ро
Counter({' ': 105213, 'о': 61282, 'а': 45209, 'е': 42519, 'и': 35838, 'н': 35119, 'т': 30619, 'с': 28128, 'л': 27277, 'в': 24824, 'р': 24570, 'к': 19328, 'д': 16387, 'м': 15940, 'у': 15454, 'п': 13847, 'я': 12477, 'г': 11177, 'ь': 10498, 'ы': 10233, 'з': 9602, 'б': 9310, 'ч': 7349, 'й': 6210, 'ж': 5460, 'ш': 5090, 'х': 4600, 'ю': 3495, 'ц': 2179, 'э': 1629, 'щ': 1514, 'ф': 1209, 'ъ': 283})
CPU times: user 151 ms, sys: 7.15 ms, total: 158 ms
Wall time: 165 ms


In [43]:
class TextEncode:
    """class for ceaser encoding"""
    def __init__(self, shift: int = 1, ru: bool = True) -> None:
        self.alphabet_ru = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
        self.shift = shift % 33 if ru else shift % 26

        self.encode_dict = {
            origin_letter: self.alphabet_ru[
                (self.alphabet_ru.index(origin_letter) + self.shift) % 33
            ]
            for origin_letter in self.alphabet_ru
        }
        self.encode_dict[" "] = " "
        self.decode_dict = {v: k for k, v in self.encode_dict.items()}

    def encode(self, text: str) -> str:
        import pandas as pd

        text_pandas = pd.Series(list(text)).replace(self.encode_dict)
        text_pandas = "".join(list(text_pandas))

        return text_pandas

    def decode(self, text: str) -> str:
        import pandas as pd

        text_pandas = pd.Series(list(text)).replace(self.decode_dict)
        text_pandas = "".join(list(text_pandas))

        return text_pandas


In [46]:
Kolmogorov_wiki = """Андре́й Никола́евич Колмого́ров (12 (25) апреля 1903, Тамбов — 20 октября 1987, Москва) — советский математик, один из крупнейших математиков XX века. Один из основоположников современной теории вероятностей, им получены фундаментальные результаты в топологии, геометрии, математической логике, классической механике, теории турбулентности, теории сложности алгоритмов, теории информации, теории функций, теории тригонометрических рядов, теории меры, теории приближения функций, теории множеств, теории дифференциальных уравнений, теории динамических систем, функциональном анализе и в ряде других областей математики и её приложений. Автор новаторских работ по философии, истории, методологии и преподаванию математики, известны его работы в статистической физике (в частности, уравнение Джонсона — Мела — Аврами — Колмогорова).

Профессор Московского государственного университета (с 1931), доктор физико-математических наук, академик Академии наук СССР (1939). Президент Московского математического общества (ММО) в 1964—1966 и 1974—1985 годах. Герой Социалистического Труда (1963). Лауреат Ленинской и Сталинской премии.

Иностранный член Национальной академии наук США (1967)[8], Лондонского королевского общества (1964)[9], Французской (Парижской) академии наук (1966)[10], член Германской академии естествоиспытателей «Леопольдина» (1959), почётный член Американской академии искусств и наук (1959), иностранный член Венгерской академии наук (1965), Польской академии наук (1956), Нидерландской королевской академии наук (1963), АН ГДР (1977), Академии наук Финляндии (1985), почётный член Румынской академии. Член Лондонского математического общества (1962), Индийского математического общества (1962), иностранный член Американского философского общества (1961). Почётный доктор Парижского университета (1955), Стокгольмского университета (1960), Индийского статистического института  (англ.)рус. в Калькутте (1962)."""

Kolmogorov_wiki_processed = TextPreproccesing(Kolmogorov_wiki).count_letters_freq()
Kolmogorov_wiki_encoded = TextEncode().encode(Kolmogorov_wiki_processed.original_text)
print(
    f"статья википедии с препроцессингом\n{Kolmogorov_wiki_processed.original_text[:50]}"
)
print("--"* 25)
print(
    f"зашифрованная статья википедии с препроцессингом\n{Kolmogorov_wiki_encoded[:50]}"
)


статья википедии с препроцессингом
андре й никола евич колмого ров апреля тамбов октя
--------------------------------------------------
зашифрованная статья википедии с препроцессингом
боесё к ойлпмб ёгйш лпмнпдп спг брсёма убнвпг плуа
